In [26]:
from google.colab import drive   #connecting to the google drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [27]:
import sqlite3
import pandas as pd

def create_database(db_var):
    connection = sqlite3.connect(db_var)
    connection.close()

def create_day_book_table(db_var):
    connection = sqlite3.connect(db_var)
    crsr = connection.cursor()
    sql_command = """
    CREATE TABLE IF NOT EXISTS day_book
    (
        entry_id INTEGER PRIMARY KEY AUTOINCREMENT,
        date DATE,
        particulars VARCHAR(20),
        vchtype VARCHAR(30),
        vhno VARCHAR(30),
        debit_amount INT(30) NULL,
        credit_amount INT(30) NULL
    );
    """
    crsr.execute(sql_command)
    connection.commit()
    connection.close()

def insert_entry(date, particulars, vchtype, vhno, debit_amount=None, credit_amount=None, db_var=None):
    connection = sqlite3.connect(db_var)
    crsr = connection.cursor()
    sql_command = """
    INSERT INTO day_book (date, particulars, vchtype, vhno, debit_amount, credit_amount)
    VALUES (?, ?, ?, ?, ?, ?)
    """
    values = (date, particulars, vchtype, vhno, debit_amount, credit_amount)
    crsr.execute(sql_command, values)
    connection.commit()
    connection.close()


def fetch_entries(db_var):
    connection = sqlite3.connect(db_var)
    cursor = connection.cursor()
    cursor.execute("SELECT * FROM day_book")
    rows = cursor.fetchall()
    connection.close()

    entries = []
    for row in rows:
        entry = {
            'entry_id': row[0],
            'date': row[1],
            'particulars': row[2],
            'vchtype': row[3],
            'vhno': row[4],
            'debit_amount': row[5],
            'credit_amount': row[6]
        }
        entries.append(entry)

    return entries

def delete_entry(entry_id, db_var):
    connection = sqlite3.connect(db_var)
    crsr = connection.cursor()
    sql_command = "DELETE FROM day_book WHERE entry_id = ?"
    crsr.execute(sql_command, (entry_id,))
    connection.commit()
    connection.close()


In [28]:
!pip install flask-ngrok        #installing Flask ngrock

In [29]:
! wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
! unzip ngrok-stable-linux-amd64.zip
! ./ngrok authtoken 2NjAFaec9sHsfPsGJFylrt4AFmy_vDSWbkijbjaB5jfuYtAx

--2023-07-06 09:14:24--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 52.202.168.65, 54.237.133.81, 54.161.241.46, ...
Connecting to bin.equinox.io (bin.equinox.io)|52.202.168.65|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13921656 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip.3’

ngrok-stable-linux- 100%[===================>]  13.28M  54.9MB/s    in 0.2s    

2023-07-06 09:14:25 (54.9 MB/s) - ‘ngrok-stable-linux-amd64.zip.3’ saved [13921656/13921656]

Archive:  ngrok-stable-linux-amd64.zip
replace ngrok? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [31]:
from flask import Flask, render_template,request,redirect,url_for
from flask_ngrok import run_with_ngrok
import sqlite3

app = Flask(__name__)
app = Flask(__name__,template_folder="/content/drive/MyDrive/TallyProject") #creating flask app

run_with_ngrok(app)  # Running with ngrok to get a single link to see the local host

db_var = "/content/drive/MyDrive/TallyProject/myTable.db"  # db location pointer

@app.route('/')
def tally_entries():
    entries = fetch_entries(db_var)

    def calculate_total_revenue(entries):
        total_revenue = 0
        for entry in entries:
            debit_amount = entry['debit_amount'] or 0  # If debit_amount is None, use 0 as default
            credit_amount = entry['credit_amount'] or 0  # If credit_amount is None, use 0 as default
            total_revenue += credit_amount - debit_amount
        return total_revenue

    def calculate_total_debit(entries):
        total_debit = 0
        for entry in entries:
            debit_amount = entry['debit_amount'] or 0  # If debit_amount is None, use 0 as default
            total_debit += debit_amount
        return total_debit

    def calculate_total_credit(entries):
        total_credit = 0
        for entry in entries:
            credit_amount = entry['credit_amount'] or 0  # If credit_amount is None, use 0 as default
            total_credit += credit_amount
        return total_credit

    total_revenue = calculate_total_revenue(entries)
    total_debit = calculate_total_debit(entries)
    total_credit = calculate_total_credit(entries)

    return render_template('Day_Book.html', entries=entries, total_revenue=total_revenue,
                           total_debit=total_debit, total_credit=total_credit)


@app.route('/insert_entry', methods=['GET', 'POST'])
def insert_entry_route():
    if request.method == 'POST':
        try:
            date = request.form['date']
            particulars = request.form['particulars']
            vchtype = request.form['vchtype']
            vhno = request.form['vhno']
            debit_amount = request.form['debit_amount']
            credit_amount = request.form['credit_amount']
            insert_entry(date, particulars, vchtype, vhno, debit_amount, credit_amount, db_var)
            return redirect(url_for('tally_entries'))
        except Exception as e:
            print(f"Error occurred during insert operation: {str(e)}")
            return "Error occurred during insert operation"
    else:
        return render_template('insert_entry.html')



@app.route('/delete_entry', methods=['GET', 'POST'])
def delete_entry_route():
    if request.method == 'POST':
        entry_id = request.form['entry_id']  # Get the entry_id from the form
        delete_entry(entry_id, db_var)
        return redirect(url_for('tally_entries'))
    else:
        return render_template('delete_entry.html')


if __name__ == '__main__':
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://37a8-35-231-48-242.ngrok-free.app
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [06/Jul/2023 09:15:48] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [06/Jul/2023 09:15:48] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [06/Jul/2023 09:16:25] "GET /insert_entry HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [06/Jul/2023 09:16:41] "POST /insert_entry HTTP/1.1" 302 -
INFO:werkzeug:127.0.0.1 - - [06/Jul/2023 09:16:41] "GET / HTTP/1.1" 200 -
